# LCviz

Load the Kepler 30 minute ("long") cadence light curveof HAT-P-11 from Kepler Quarter 9 (in 2011). HAT-P-11 is a K4V host to a transiting hot Neptune with a 4.8 day period, and a stellar rotation period of 29 days.

In [ ]:
from lightkurve import search_lightcurve
from lcviz import LCviz

lc_q9 = search_lightcurve("HAT-P-11", mission="Kepler", cadence="long", quarter=9).download()

lcviz = LCviz()
lcviz.load_data(lc_q9)
lcviz.show()

Load and display a second Kepler quarter in the same viewer. We'll pre-process the light curves by flattening them to remove out-of-transit variability due to stellar rotation:

In [ ]:
lc_q10 = search_lightcurve("HAT-P-11", mission="Kepler", cadence="long", quarter=10).download()

def preprocess_lc(lc, window_length=50):
    """
    Here we use the lightkurve `flatten` method for roughly detrending
    the light curve. There will be workflows for doing this operation
    through the UI soon.
    """
    return lc.flatten(
        window_length=window_length, polyorder=3
    ).remove_nans()

lcviz = LCviz()
lcviz.load_data(preprocess_lc(lc_q9), data_label='Q9')
lcviz.load_data(preprocess_lc(lc_q10), data_label='Q10')
lcviz.show()

Show a phase folded light curve, given the ephemeris for the transiting exoplanet HAT-P-11 b:

In [ ]:
import astropy.units as u
from astropy.time import Time

# origin on the time axis
time_coord = lcviz.app.data_collection[0].coords
reference_time = time_coord.reference_time

# literature ephemeris for hot Neptune planet HAT-P-11 b
morris2017_epoch = Time(2454605.89146, format='jd')
morris2017_period = 4.88780258  # days

# phase the transit light curve
eph = lcviz.plugins['Ephemeris']
eph.period = morris2017_period
eph.t0 = (
    (morris2017_epoch - reference_time).to_value(time_coord.unit) % eph.period
)
# recenter the phase axis with transit at phase=0.5
eph.t0 += 0.5 * eph.period